In [124]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import Series, DataFrame
import time

In [125]:
url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13106&sc=13107&sc=13108&sc=13118&sc=13121&sc=13122&sc=13123&sc=13109&sc=13110&sc=13111&sc=13112&sc=13114&sc=13115&sc=13120&sc=13116&sc=13117&sc=13119&cb=0.0&ct=9999999&mb=0&mt=9999999&et=9999999&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=25&pc=50"

result = requests.get(url)
content = result.content
soup = BeautifulSoup(content)
soup

<!DOCTYPE html>

<!--[if lt IE 7]><html lang="ja" class="ie"><![endif]-->
<!--[if IE 7]><html lang="ja" class="ie"><![endif]-->
<!--[if IE 8]><html lang="ja" class="ie"><![endif]-->
<!--[if gt IE 8]><!--><html lang="ja"><!--<![endif]-->
<!-- FR301FC0012 START -->
<head>
<meta charset="utf-8"/>
<!-- seo.headerTag Start -->
<title>【SUUMO】東京都千代田区、東京都中央区、東京都港区、東京都新宿区、東京都文京区、東京都渋谷区、東京都台東区、東京都墨田区、東京都江東区、東京都荒川区、東京都足立区、東京都葛飾区、東京都江戸川区、東京都品川区、東京都目黒区、東京都大田区、東京都世田谷区、東京都中野区、東京都杉並区、東京都練馬区、東京都豊島区、東京都北区、東京都板橋区の賃貸(賃貸マンション・アパート)住宅のお部屋探し物件情報</title>
<meta content="千代田区,中央区,港区,新宿区,文京区,台東区,墨田区,江東区,品川区,目黒区,大田区,世田谷区,渋谷区,中野区,杉並区,豊島区,北区,荒川区,板橋区,練馬区,足立区,葛飾区,江戸川区,東京都,賃貸,賃貸マンション,アパート,賃貸アパート,賃貸情報,部屋探し,賃貸住宅,物件,賃貸物件,住まい,SUUMO,スーモ" name="keywords"/>
<meta content="【SUUMO(スーモ)賃貸】東京都千代田区、東京都中央区、東京都港区、東京都新宿区、東京都文京区、東京都渋谷区、東京都台東区、東京都墨田区、東京都江東区、東京都荒川区、東京都足立区、東京都葛飾区、東京都江戸川区、東京都品川区、東京都目黒区、東京都大田区、東京都世田谷区、東京都中野区、東京都杉並区、東京都練馬区、東京都豊島区、東京都北区、東京都板橋区でアパート・賃貸マンションを借りるならリクルートの賃貸情報サイトSUUMO(スーモ)にお任せください。全国の賃貸住宅・貸家の物件情報からさまざまな条件を指定して検索

In [213]:
house_list = soup.find('div', {'id': 'js-bukkenList'})
cassette_items = house_list.find_all('div', {'class': 'cassetteitem'})
a = cassette_items[0]
b = a.find_all('div', {'class', 'cassetteitem_detail-text'})
b[0].text.split()[1]

'歩7分'

In [126]:
pages_set = soup.find('div', {'class': 'pagination_set-nav'}).text
pages_split = pages_set.split()
page = int(pages_split[5])

In [167]:
urls = [url]

for num in range(2, page+1):
    page_url = url + '&=' + str(num)
    urls.append(page_url)

In [181]:
name = []  # 建物名
address = []  # 住所
station1 = []  # 最寄り
station2 = []
station3 = []
age = []  # 築年数

floor = []  # 階数
rent = []  # 賃料
admin = []  # 管理費
plan = []  # 間取り
area = []  # 面積
deposit = []  # 敷金
gratuity = []  # 礼金

In [217]:
for url in urls:
    result = requests.get(url)
    content = result.content
    soup = BeautifulSoup(content)
    house_list = soup.find('div', {'id': 'js-bukkenList'})
    cassette_items = house_list.find_all('div', {'class': 'cassetteitem'})

    for cassette in cassette_items:
        n_room = len(cassette.find_all('tbody'))
        house_name = str(cassette.find('div', {'class': 'cassetteitem_content-title'}).text)
        house_address = str(cassette.find('li', {'class': 'cassetteitem_detail-col1'}).text)
        station_list = cassette.find_all('div', {'class', 'cassetteitem_detail-text'})
        if len(station_list) != 3:
            for i in range(3-len(station_list)):
                station_list.append('None/None -1')
        text_age = cassette.find('li', 'cassetteitem_detail-col3').text.strip().split()[0]
        if text_age == '新築':
            house_age = 0
        else:
            house_age = int(text_age.replace('築', '').replace('年', ''))
        
        for i in range(n_room):
            name.append(house_name)
            address.append(house_address)
            station1.append((station_list[0].text.split()[0].split('/')[1], int(station_list[0].text.split()[1].replace('歩', '').replace('バス', '').replace('分', ''))))
            station2.append((station_list[1].text.split()[0].split('/')[1], int(station_list[1].text.split()[1].replace('歩', '').replace('バス', '').replace('分', ''))))
            station3.append((station_list[2].text.split()[0].split('/')[1], int(station_list[2].text.split()[1].replace('歩', '').replace('バス', '').replace('分', ''))))
            age.append(house_age)
        
        body = cassette.find_all('tbody')
        
        for i in range(n_room):
            room_info = body[i].find_all('td')
            floor.append(int(room_info[2].text.strip()[0]))
            rent.append(float(room_info[3].find_all('li')[0].text.strip('万円')))
            temp = room_info[3].find_all('li')[1].text.strip('円')
            if temp == '-':
                temp = 0
            admin.append(int(temp))
            temp = room_info[4].find_all('li')[0].text.strip('万円')
            if temp == '-':
                temp = 0
            deposit.append(float(temp))
            temp = room_info[4].find_all('li')[1].text.strip('万円')
            if temp == '-':
                temp = 0
            gratuity.append(float(temp))
            plan.append(room_info[5].find_all('li')[0].text)
            area.append(float(room_info[5].find_all('li')[1].text.strip('m2')))
            
    break
        
time.sleep(1)

IndexError: list index out of range

In [177]:
print(len(name), len(address), len(station1), len(station2), len(station3), len(age))
print(len(floor), len(rent), len(admin), len(plan), len(area), len(deposit), len(gratuity))
station3

276 276 276 276 272 272
272 272 272 272 272 272 272


[('南千住駅', 7),
 ('三ノ輪駅', 6),
 ('尾久駅', 5),
 ('尾久駅', 5),
 ('町屋駅', 12),
 ('尾久駅', 6),
 ('尾久駅', 6),
 ('日暮里駅', 6),
 ('日暮里駅', 6),
 ('日暮里駅', 6),
 ('日暮里駅', 6),
 ('日暮里駅', 6),
 ('日暮里駅', 6),
 ('日暮里駅', 6),
 ('日暮里駅', 6),
 ('日暮里駅', 6),
 ('お花茶屋駅', 7),
 ('お花茶屋駅', 7),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 7),
 ('金町駅', 7),
 ('金町駅', 7),
 ('堀切菖蒲園駅', 9),
 ('京成小岩駅', 8),
 ('京成小岩駅', 8),
 ('京成小岩駅', 8),
 ('京成小岩駅', 8),
 ('亀有駅', 7),
 ('お花茶屋駅', 8),
 ('お花茶屋駅', 8),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('日暮里駅', 5),
 ('三ノ輪駅', 4),
 ('